# SH3GL2 - Single gene analysis in GP2 data

## Description

Using "individual level data"

### 0. Getting Started

- Loading Python libraries
- Defining functions
- Installing packages

### 1. Copy data from workspace to cloud environment

### 2. Extract SH3GL2

### 3. Annotate SH3GL2 variants

### 4. Extract coding/non-syn variants

### 5. Calculate frequency in cases versus controls

### 6. Calculate frequency (homozygotes) in cases versus controls

### 7. Save out results

## Getting Started

### Loading Python libraries

### Defining functions

In [1]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

In [2]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

### Set paths

In [3]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name: {WORKSPACE_NAME}')
print(f'Billing Project: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data: {WORKSPACE_BUCKET}')
print('')


## GP2 v5.0
## Explicitly define release v5.0 path 
GP2_RELEASE_PATH = 'gs://gp2tier2/release5_11052023'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_META_RELEASE_PATH = f'{GP2_RELEASE_PATH}/meta_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v5.0')
print(f'Path to GP2 v5.0 Clinical Data: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v5.0 Raw Genotype Data: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v5.0 Imputed Genotype Data: {GP2_IMPUTED_GENO_PATH}')


## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'gs://amp-pd-data/releases/2022_v3release_1115'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_PLINK_PFILES = os.path.join(AMP_WGS_RELEASE_PLINK_PATH, 'pfiles')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

Billing and Workspace
Workspace Name: Endophilin-A
Billing Project: terra-18d8e41c
Workspace Bucket, where you can upload and download data: gs://fc-f7a400c1-827e-48f8-b7b6-90c488a000a4

GP2 v5.0
Path to GP2 v5.0 Clinical Data: gs://gp2tier2/release5_11052023/clinical_data
Path to GP2 v5.0 Raw Genotype Data: gs://gp2tier2/release5_11052023/raw_genotypes
Path to GP2 v5.0 Imputed Genotype Data: gs://gp2tier2/release5_11052023/imputed_genotypes
AMP-PD v3.0
Path to AMP-PD v3.0 Clinical Data: gs://amp-pd-data/releases/2022_v3release_1115/clinical
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink/pfiles



### Install packages

#### Install Plink 1.9 and Plink 2.0

In [4]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi


if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220603.zip
unzip -o plink2_linux_avx2_20220603.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/
Plink2 is already installed in /home/jupyter/tools/


In [5]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Remote restrictions

In [6]:
%%bash

# chmod plink 1.9 
chmod u+x /home/jupyter/tools/plink

In [7]:
%%bash

# chmod plink 2.0
chmod u+x /home/jupyter/tools/plink2

### Install ANNOVAR

In [8]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is already installed in /home/jupyter/tools/


In [9]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Install ANNOVAR: Download sources of annotation 

In [10]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished d

In [11]:
%%bash
ls /home/jupyter/tools/annovar/

annotate_variation.pl
coding_change.pl
convert2annovar.pl
example
humandb
retrieve_seq_from_fasta.pl
table_annovar.pl
variants_reduction.pl


## EUR

## Copy data from GP2 bucket to workspace

In [4]:
# Make a directory
print("Making a working directory")
WORK_DIR = f'/home/jupyter/SH3GL2_GP2/'
shell_do(f'mkdir -p {WORK_DIR}') # f' means f-string - contains expressions to execute the code

Making a working directory


Executing: mkdir -p /home/jupyter/SH3GL2_GP2/


In [5]:
# Check directory where GP2 data is
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_IMPUTED_GENO_PATH}/EUR/')

Executing: gsutil -u terra-18d8e41c ls gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/


gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr10_EUR_release5.log
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr10_EUR_release5.pgen
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr10_EUR_release5.psam
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr10_EUR_release5.pvar
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr10_EUR_release5_no_mac_filter.log
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr10_EUR_release5_no_mac_filter.pgen
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr10_EUR_release5_no_mac_filter.psam
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr10_EUR_release5_no_mac_filter.pvar
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr11_EUR_release5.log
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr11_EUR_release5.pgen
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr11_EUR_release5.psam
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/chr11_EUR_release5.pvar
gs://gp2tie

In [ ]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/EUR/chr9_EUR_release5_no_mac_filter* {WORK_DIR}')

### Create a covariate file with GP2 data

In [ ]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_CLINICAL_RELEASE_PATH}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_CLINICAL_RELEASE_PATH}/master_key_release5_final.csv {WORK_DIR}')

In [ ]:
cov = pd.read_csv(f'{WORK_DIR}/master_key_release5_final.csv')
cov.columns

In [ ]:
cov_reduced = cov[['GP2sampleID','GP2sampleID','sex_for_qc', 'age', 'phenotype']]
cov_reduced.columns = ['FID','IID', 'SEX','AGE','PHENO']
#cov_reduced

In [ ]:
conditions = [
    (cov_reduced['PHENO'] == "PD"),
    (cov_reduced['PHENO'] == "Control")]

In [ ]:
choices = [2,1]
cov_reduced['PHENOTYPE'] = np.select(conditions, choices, default=-9).astype(np.int64)

In [ ]:
cov_reduced.reset_index(inplace=True)
cov_reduced.drop(columns=["index"], inplace=True)

In [ ]:
cov_reduced.drop(columns=['PHENO'], inplace=True)

In [ ]:
sex = cov_reduced[['FID','IID','SEX']]
sex.to_csv(f'{WORK_DIR}/SEX.txt',index=False, sep="\t")

In [ ]:
pheno = cov_reduced[['FID','IID','PHENOTYPE']]
pheno.to_csv(f'{WORK_DIR}/PHENO.txt',index=False, sep="\t")

## Extract SH3GL2

In [ ]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

# SH3GL2: gene positions on hg38 (from https://useast.ensembl.org/index.html)
/home/jupyter/tools/plink2 \
--pfile chr9_EUR_release5_no_mac_filter \
--chr 9 \
--from-bp 17579066  \
--to-bp 17797124 \
--make-bed \
--out pheno_SH3GL2_EUR

### Visualize plink files bim, fam and bed

In [ ]:
%%bash

# Visualize bim file
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

head pheno_SH3GL2_EUR.bim

In [ ]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

head pheno_SH3GL2_EUR.fam

In [ ]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

# Turn binary files into VCF
/home/jupyter/tools/plink \
--bfile pheno_SH3GL2_EUR \
--recode vcf-fid \
--out pheno_SH3GL2_EUR_nomacfilter

## Annotate SH3GL2 variants

In [ ]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR
export PATH=$PATH:/home/jupyter/tools/rvtests/third/tabix-0.2.6/

### Bgzip and Tabix
bgzip pheno_SH3GL2_EUR_nomacfilter.vcf

tabix -f -p vcf pheno_SH3GL2_EUR_nomacfilter.vcf.gz

In [ ]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

### Annotate variants using ANNOVAR: https://annovar.openbioinformatics.org/en/latest/ 
perl /home/jupyter/tools/annovar/table_annovar.pl pheno_SH3GL2_EUR_nomacfilter.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out pheno_SH3GL2_EUR.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput

## Extract coding and non-syn variants

In [5]:
# Visualize multianno file
SH3GL2_EUR = pd.read_csv(f'{WORK_DIR}/pheno_SH3GL2_EUR.annovar.hg38_multianno.txt', sep = '\t')

In [7]:
SH3GL2_EUR['Chr_bp'] = SH3GL2_EUR['Chr'].astype(str) + ':' + SH3GL2_EUR['Start'].astype(str) + ':' + SH3GL2_EUR['End'].astype(str)

In [8]:
SH3GL2_EUR = SH3GL2_EUR.drop_duplicates(subset=['Chr_bp'])

In [9]:
SH3GL2_EUR = SH3GL2_EUR[SH3GL2_EUR['Gene.refGene']=='SH3GL2']

In [10]:
#Number of total varints 
SH3GL2_EUR.count()

Chr               17368
Start             17368
End               17368
Ref               17368
Alt               17368
                  ...  
Otherinfo15365    17368
Otherinfo15366    17368
Otherinfo15367    17368
Otherinfo15368    17368
Chr_bp            17368
Length: 15380, dtype: int64

In [11]:
reduced_SH3GL2_EUR = SH3GL2_EUR[["Chr_bp", "Gene.refGene", 'Func.refGene','ExonicFunc.refGene']]
reduced_SH3GL2_EUR

,Chr_bp,Gene.refGene,Func.refGene,ExonicFunc.refGene
0,9:17579077:17579077,SH3GL2,UTR5,.
1,9:17579083:17579083,SH3GL2,UTR5,.
2,9:17579085:17579085,SH3GL2,UTR5,.
3,9:17579086:17579086,SH3GL2,UTR5,.
4,9:17579090:17579090,SH3GL2,UTR5,.
...,...,...,...,...
17864,9:17797008:17797008,SH3GL2,UTR3,.
17865,9:17797011:17797011,SH3GL2,UTR3,.
17866,9:17797047:17797047,SH3GL2,UTR3,.
17867,9:17797079:17797079,SH3GL2,UTR3,.


In [12]:
reduced_SH3GL2_EUR.to_csv(f'{WORKSPACE_BUCKET}/reduced_SH3GL2_EUR.txt', sep = '\t', index = False, header= True)

In [10]:
SH3GL2_EUR['Func.refGene'].value_counts()

intronic    17164
UTR3          122
exonic         48
UTR5           34
Name: Func.refGene, dtype: int64

In [11]:
SH3GL2_EUR['ExonicFunc.refGene'].value_counts()

.                    17320
nonsynonymous SNV       30
synonymous SNV          18
Name: ExonicFunc.refGene, dtype: int64

In [12]:
# Filter exonic variants
coding_EUR = SH3GL2_EUR[SH3GL2_EUR['Func.refGene'] == 'exonic']
coding_EUR.count()

Chr               48
Start             48
End               48
Ref               48
Alt               48
                  ..
Otherinfo15365    48
Otherinfo15366    48
Otherinfo15367    48
Otherinfo15368    48
Chr_bp            48
Length: 15380, dtype: int64

In [13]:
# Filter exonic and non-syn vars
coding_nonsynonymous_EUR = SH3GL2_EUR[(SH3GL2_EUR['Func.refGene'] == 'exonic') & (SH3GL2_EUR['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous_EUR

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo15360,Otherinfo15361,Otherinfo15362,Otherinfo15363,Otherinfo15364,Otherinfo15365,Otherinfo15366,Otherinfo15367,Otherinfo15368,Chr_bp
13861,9,17747115,17747115,A,C,exonic,SH3GL2,.,nonsynonymous SNV,SH3GL2:NM_003026:exon2:c.A95C:p.D32A,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,9:17747115:17747115
15037,9,17761500,17761500,C,G,exonic,SH3GL2,.,nonsynonymous SNV,SH3GL2:NM_003026:exon3:c.C178G:p.P60A,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,9:17761500:17761500
17096,9,17787498,17787498,T,A,exonic,SH3GL2,.,nonsynonymous SNV,SH3GL2:NM_003026:exon5:c.T450A:p.D150E,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,9:17787498:17787498
17231,9,17789425,17789425,G,T,exonic,SH3GL2,.,nonsynonymous SNV,SH3GL2:NM_003026:exon6:c.G499T:p.D167Y,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,9:17789425:17789425
17234,9,17789473,17789473,C,G,exonic,SH3GL2,.,nonsynonymous SNV,SH3GL2:NM_003026:exon6:c.C547G:p.L183V,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,9:17789473:17789473
17235,9,17789499,17789499,T,G,exonic,SH3GL2,.,nonsynonymous SNV,SH3GL2:NM_003026:exon6:c.T573G:p.D191E,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,9:17789499:17789499
17236,9,17789545,17789545,A,G,exonic,SH3GL2,.,nonsynonymous SNV,SH3GL2:NM_003026:exon6:c.A619G:p.M207V,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,9:17789545:17789545
17386,9,17791272,17791272,G,C,exonic,SH3GL2,.,nonsynonymous SNV,SH3GL2:NM_003026:exon7:c.G666C:p.Q222H,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,9:17791272:17791272
17387,9,17791276,17791276,G,C,exonic,SH3GL2,.,nonsynonymous SNV,SH3GL2:NM_003026:exon7:c.G670C:p.E224Q,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,9:17791276:17791276
17388,9,17791290,17791290,G,C,exonic,SH3GL2,.,nonsynonymous SNV,SH3GL2:NM_003026:exon7:c.G684C:p.Q228H,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,9:17791290:17791290


In [14]:
coding_nonsynonymous_EUR.to_csv(f'{WORK_DIR}/coding_nonsynonymous.txt', sep = '\t', index = False)

In [15]:
# Calculate freq - cases vs controls

In [16]:
reduced_coding_nonsynonymous_EUR = coding_nonsynonymous_EUR[["Chr", "Start", "End", "Gene.refGene"]]
reduced_coding_nonsynonymous_EUR.to_csv(f'{WORK_DIR}/reduced_coding_nonsynonymous_EUR.txt', sep = '\t', index = False, header= False)

In [17]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_SH3GL2_EUR --extract range reduced_coding_nonsynonymous_EUR.txt --assoc --ci 0.95 --out coding_nonsynonymous_pheno_SH3GL2_EUR --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_SH3GL2_EUR.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile pheno_SH3GL2_EUR
  --ci 0.95
  --extract range reduced_coding_nonsynonymous_EUR.txt
  --out coding_nonsynonymous_pheno_SH3GL2_EUR

14998 MB RAM detected; reserving 7499 MB for main workspace.
17869 variants loaded from .bim file.
15356 people (9460 males, 5896 females) loaded from .fam.
14196 phenotype values loaded from .fam.
--extract range: 17839 variants excluded.
--extract range: 30 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 15356 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%

In [18]:
SH3GL2_EUR_freq = pd.read_csv(f'{WORK_DIR}coding_nonsynonymous_pheno_SH3GL2_EUR.assoc', delim_whitespace=True)
SH3GL2_EUR_freq

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR,SE,L95,U95
0,9,chr9:17747115:A:C,17747115,C,0.000000,0.000000,A,NaN,NaN,NaN,NaN,NaN,NaN
1,9,chr9:17761500:C:G,17761500,G,0.000108,0.000101,C,0.003584,0.9523,1.076,1.2250,0.09756,11.870
2,9,chr9:17787498:T:A,17787498,A,0.000162,0.000101,T,0.175300,0.6754,1.614,1.1550,0.16790,15.520
3,9,chr9:17789425:G:T,17789425,T,0.000000,0.000000,G,NaN,NaN,NaN,NaN,NaN,NaN
4,9,chr9:17789473:C:G,17789473,G,0.000000,0.000000,C,NaN,NaN,NaN,NaN,NaN,NaN
5,9,chr9:17789499:T:G,17789499,G,0.000000,0.000000,T,NaN,NaN,NaN,NaN,NaN,NaN
6,9,chr9:17789545:A:G,17789545,G,0.000054,0.000000,A,0.538000,0.4632,NaN,NaN,NaN,NaN
7,9,chr9:17791272:G:C,17791272,C,0.000000,0.000000,G,NaN,NaN,NaN,NaN,NaN,NaN
8,9,chr9:17791276:G:C,17791276,C,0.000000,0.000000,G,NaN,NaN,NaN,NaN,NaN,NaN
9,9,chr9:17791290:G:C,17791290,C,0.000054,0.000000,G,0.538000,0.4632,NaN,NaN,NaN,NaN


In [20]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_SH3GL2_EUR --extract range reduced_coding_nonsynonymous_EUR.txt --recode A --out coding_nonsynonymous_pheno_SH3GL2_EUR

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_SH3GL2_EUR.log.
Options in effect:
  --bfile pheno_SH3GL2_EUR
  --extract range reduced_coding_nonsynonymous_EUR.txt
  --out coding_nonsynonymous_pheno_SH3GL2_EUR
  --recode A

14998 MB RAM detected; reserving 7499 MB for main workspace.
17869 variants loaded from .bim file.
15356 people (9460 males, 5896 females) loaded from .fam.
14196 phenotype values loaded from .fam.
--extract range: 17839 variants excluded.
--extract range: 30 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 15356 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34

In [21]:
SH3GL2_EUR_recode = pd.read_csv(f'{WORK_DIR}coding_nonsynonymous_pheno_SH3GL2_EUR.raw', delim_whitespace=True)
SH3GL2_EUR_recode.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:17747115:A:C_C,chr9:17761500:C:G_G,chr9:17787498:T:A_A,chr9:17789425:G:T_T,...,chr9:17793471:C:T_T,chr9:17793477:C:T_T,chr9:17795552:A:G_G,chr9:17795571:G:A_A,chr9:17795582:G:A_A,chr9:17795624:G:A_A,chr9:17795649:A:G_G,chr9:17795682:A:G_G,chr9:17795685:G:A_A,chr9:17795705:A:G_G
0,0,APGS_000001_s1,0,0,1,2,0,0,0.0,0,...,0.0,0,0.0,0,0,0,0,0,0,0.0
1,0,APGS_000002_s1,0,0,1,2,0,0,0.0,0,...,0.0,0,0.0,0,0,0,0,0,0,0.0
2,0,APGS_000003_s1,0,0,1,2,0,0,0.0,0,...,0.0,0,0.0,0,0,0,0,0,0,0.0
3,0,APGS_000004_s1,0,0,2,2,0,0,0.0,0,...,0.0,0,0.0,0,0,0,0,0,0,0.0
4,0,APGS_000005_s1,0,0,1,2,0,0,0.0,0,...,0.0,0,0.0,0,0,0,0,0,0,0.0


## Save out results..!

In [62]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR} {WORKSPACE_BUCKET}')

Executing: gsutil -mu terra-18d8e41c cp -r /home/jupyter/SH3GL2_GP2/ gs://fc-f7a400c1-827e-48f8-b7b6-90c488a000a4


Copying file:///home/jupyter/SH3GL2_GP2/pheno_SH3GL2_AFR.vcf.gz [Content-Type=text/vcard]...
Copying file:///home/jupyter/SH3GL2_GP2/chr9_AAC_release5.psam [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/SH3GL2_GP2/chr9_AJ_release5.pgen [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/SH3GL2_GP2/pheno_SH3GL2_AFR.bed [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/SH3GL2_GP2/chr9_FIN_release5.log [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
comp